In [20]:
%pip install beautifulsoup4
import pandas as pd
import numpy as np
import random 
from bs4 import BeautifulSoup
import requests 

Note: you may need to restart the kernel to use updated packages.


In [4]:
!conda update -n base -c defaults conda --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-package-handling-1.3.11|           py37_0         260 KB
    certifi-2019.6.16          |           py37_1         156 KB
    libxml2-2.9.9              |       hea5a465_1         2.0 MB
    ca-certificates-2019.5.15  |                1         134 KB
    chardet-3.0.4              |        py37_1003         173 KB
    python-3.7.4               |       h265db76_1        36.5 MB
    pysocks-1.7.0              |           py37_0          29 KB
    python-libarchive-c-2.8    |          py37_13          23 KB
    pip-19.2.2                 |           py37_0         1.9 MB
    conda-4.7.11               |           py37_0         3.0 MB
    ncurses-6.1                |       he6710b0_1         958 KB
    

In [5]:
%pip install requests
import requests
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'html')


Note: you may need to restart the kernel to use updated packages.


In [6]:
My_table = soup.find('table', {"class": 'wikitable sortable'})

In [7]:
New_Table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# Extracting a clean form of the table
for tr_cell in New_Table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [8]:
Unique_Postal = set(Postcode)
print('num of unique Postal codes:', len(Unique_Postal))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in Unique_Postal:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [9]:
Toronto_Dictionary = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_Toronto = pd.DataFrame.from_dict(Toronto_Dictionary)
df_Toronto.to_csv('Toronto_Part-A.csv')
df_Toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9L,North York,Humber Summit
4,M4Y,Downtown Toronto,Church and Wellesley
5,M9N,York,Weston
6,M3J,North York,"Northwood Park, York University"
7,M2H,North York,Hillcrest Village
8,M2J,North York,Henry Farm
9,M5S,Downtown Toronto,University of Toronto


In [21]:
%pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [11]:
import geocoder

In [12]:
Table_PostalCode = soup.find('table')
fields = Table_PostalCode.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_PostalCode = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_PostalCode.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostalCode.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:

df_PostalCode['Borough'].replace('Not assigned', np.nan, inplace=True)
df_PostalCode.dropna(subset=['Borough'], inplace=True)

df_PostalCode.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
df_PostalCodeNeighbor = df_PostalCode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_PostalCodeNeighbor.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostalCodeNeighbor

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [15]:
df_PostalCodeNeighbor['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
df_PostalCodeNeighbor

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [16]:
df_PostalCodeNeighbor.shape

(103, 3)

In [17]:

df_geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
df_geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_Post = pd.merge(df_PostalCodeNeighbor, df_geospatial_data, on=['Postcode'], how='inner')
df_Full_Table = df_Post[[ 'Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',]].copy()
df_Full_Table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
df_Full_Table.head(103)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [22]:
%pip install geopy

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [25]:
import geocoder

In [26]:
df = pd.read_csv('Toronto_Part-A.csv')

In [27]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import requests
import folium 
print('Libraries imported.')

Libraries imported.


In [28]:
df_Toronto = pd.read_csv('Toronto_Part-A.csv')
df_Toronto.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood
0,0,M9A,Etobicoke,Islington Avenue
1,1,M4H,East York,Thorncliffe Park
2,2,M1B,Scarborough,"Rouge, Malvern"
3,3,M9L,North York,Humber Summit
4,4,M4Y,Downtown Toronto,Church and Wellesley
5,5,M9N,York,Weston
6,6,M3J,North York,"Northwood Park, York University"


In [29]:
df_geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
df_geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df_Post = pd.merge(df_Toronto, df_geospatial_data, on=['Postcode'], how='inner')
df_Toronto = df_Post[[ 'Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',]].copy()
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,M9L,North York,Humber Summit,43.756303,-79.565963
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


In [34]:
import folium
Toronto_latitude = 43.6532; Toronto_longitude = -79.3832
map_Toronto = folium.Map(location = [Toronto_latitude, Toronto_longitude], zoom_start = 10.7)
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'],
                                           df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto